# Can we monitor what7 redditors say recently?

https://github.com/pushshift/api/blob/master/README.md

https://github.com/dmarx/psaw

In [79]:
import requests
import json
import pandas as pd
from pmaw import PushshiftAPI

kw = 'GME'
subreddit='wallstreetbets'
obs_hrs = 24

curr_obs_end = pd.datetime.now().replace(minute=0, second=0, microsecond=0)
curr_obs_start = pd.datetime.timestamp(curr_obs_end - pd.Timedelta(f"{obs_hrs}h"))

prev_obs_end = curr_obs_end - pd.Timedelta("24h")
prev_obs_start = curr_obs_start - 86400

api = PushshiftAPI()
curr_gen = api.search_comments(q=kw, subreddit=subreddit, 
                               after=int(curr_obs_start), 
                               before=int(curr_obs_end.timestamp()))
curr_df = pd.DataFrame([result for result in curr_gen])

prev_gen = api.search_comments(q=kw, subreddit=subreddit, 
                               after=int(prev_obs_start), 
                               before=int(prev_obs_end.timestamp()))
prev_df = pd.DataFrame([result for result in prev_gen])

Total:: Success Rate: 100.00% - Requests: 1 - Batches: 1
6794 results available in Pushshift
Setting limit to 6794
Checkpoint:: Success Rate: 96.43% - Requests: 84 - Batches: 10
Total:: Success Rate: 96.43% - Requests: 84 - Batches: 10
Total:: Success Rate: 96.47% - Requests: 85 - Batches: 11
6794 results available in Pushshift
Total:: Success Rate: 93.26% - Requests: 89 - Batches: 12
Total:: Success Rate: 100.00% - Requests: 1 - Batches: 1
8844 results available in Pushshift
Setting limit to 8844
Checkpoint:: Success Rate: 90.11% - Requests: 91 - Batches: 10
Total:: Success Rate: 88.33% - Requests: 120 - Batches: 14
Total:: Success Rate: 88.43% - Requests: 121 - Batches: 15
8844 results available in Pushshift
Total:: Success Rate: 88.52% - Requests: 122 - Batches: 16


In [80]:
curr_df['set'] = "Today"
prev_df['set'] = "Yesterday"
prev_df['created_utc'] = prev_df['created_utc'] + 86400

df = pd.concat([curr_df, prev_df], ignore_index=True)

In [81]:
df['created_ts'] = pd.to_datetime(df['created_utc'], unit='s', utc=True)
df['created_ts'] = df['created_ts'].dt.tz_convert('Asia/Hong_Kong')

In [82]:
import plotly.express as px

hist = px.histogram(df, x='created_ts', color='set', barmode="overlay", #nbins=obs_hrs,
                    title=f"Occurrence of {kw} in r/{subreddit} in the last {obs_hrs} hours")
hist.show()